To begin with, I would like to extract all the pressures for the amines that are available in PubChem. To do this, I want to first do this for all the compounds in the amine dataset. 

In [1]:
# Import packages 
import dataLoading 
import requests
import time
import pandas as pd
from rdkit import Chem
import matplotlib.pyplot as plt
from modified_pubchemprops import get_second_layer_props
import re
import pubchempy as pcp
import numpy as np

pd.set_option('display.max_columns', None)

In [97]:
# Read the excel file into dataframe. This file contains the PubChem CID of the compounds that have data available. 
df = pd.read_excel("PubChem_has_data_available.xlsx")

df

,CID,NAME,FORMULA,InChI,SMILES
0,297,Methane,CH4,InChI=1S/CH4/h1H4,C
1,533,Hydrogen selenide,H2Se,InChI=1S/H2Se/h1H2,[SeH2]
2,588,creatinine,C4H7N3O,"InChI=1S/C4H7N3O/c1-7-2-3(8)6-4(7)5/h2H2,1H3,(...",CN1CC(=O)N=C1N
3,783,Hydrogen,H2,InChI=1S/H2/h1H,[HH]
4,807,Iodine,I2,InChI=1S/I2/c1-2,II
...,...,...,...,...,...
4760,154824931,"(8R)-1,5,6,8,9,10,11,11-octachloro-4-oxatetrac...",C10H4Cl8O,"InChI=1S/C10H4Cl8O/c11-3-1-2(6-9(3,16)19-6)8(1...",C12C(C(C3(C1O3)Cl)Cl)C4(C(=C(C2(C4(Cl)Cl)Cl)Cl...
4761,154825517,"(1R,3R,4S,6S,7S)-1,3,4,7,8,9,10,10-octachlorot...",C10H6Cl8,InChI=1S/C10H6Cl8/c11-3-1-2-4(5(3)12)9(16)7(14...,C1C2C(C(C1Cl)Cl)C3(C(=C(C2(C3(Cl)Cl)Cl)Cl)Cl)Cl
4762,154825547,"(4E)-4-ethylidene-7-hydroxy-6,7,14-trimethyl-2...",C19H27NO6,"InChI=1S/C19H27NO6/c1-5-13-10-12(2)19(3,24)18(...",CC=C1CC(C(C(=O)OCC2=CCN(CCC(C2=O)OC1=O)C)(C)O)C
4763,156619590,"(2R,3aS,5aR,5bS,9S,13S,14R,16aS,16bR)-2-((6-De...",C41H65NO10,InChI=1S/C41H65NO10/c1-10-26-12-11-13-34(52-36...,CCC1CCCC(C(C(=O)C2=CC3C4CC(CC4C=CC3C2CC(=O)O1)...


To search up the vapor pressure using the "pubchemprops", we need the CAS number of the compound and not the CID number. Therefore, I use the pubchempy

In [14]:
def extract_vapor_pressure(cid_number):
    # Extract the vapor pressure from the PubChem database
    cid_input = True
    vapor_pressure = get_second_layer_props(cid_number, ['Vapor Pressure'], cid_input)
    return vapor_pressure

#print(extract_vapor_pressure(533)["Vapor Pressure"])
print(extract_vapor_pressure(533)["Vapor Pressure"][0]["Value"])
print(extract_vapor_pressure(533)["Vapor Pressure"][1]["Value"])

{'StringWithMarkup': [{'String': '12 atm at 30.8 °C; 4.5 atm at 0.2 °C; 1.75 atm at -30 °C'}]}
{'StringWithMarkup': [{'String': 'Vapor pressure, kPa at 21 °C: 878'}]}


In [100]:
def convert_from_mmHg_to_Pa(mmHg):
    """ 
    Converts from mmHg to Pa. 
    """
    return mmHg * 133.322


def convert_from_farhenheit_to_celsius(fahrenheit):
    """
    Converts from Fahrenheit to Celsius.
    """
    return (fahrenheit - 32) * 5 / 9


def extract_temperature_from_string(text):
    """
    Extracts the temperature from a string. If the temperature is in Fahrenheit, it converts it to Celsius. If the function is unable 
    to extract the temperature, it returns None.
    """
    # Pattern for Celsius, Fahrenheit and Kelvin. 
    celsius_pattern = r"(-?\d+\.?\d*)\s*°c"
    fahrenheit_pattern = r"(-?\d+\.?\d*)\s*°f"
    kelvin_pattern = r"(-?\d+\.?\d*)\s*°k"


    # Search for Celsius
    match_celsius = re.search(celsius_pattern, text)
    if match_celsius:
        temperature_celsius = float(match_celsius.group(1))
        return temperature_celsius

    # Search for Fahrenheit
    match_fahrenheit = re.search(fahrenheit_pattern, text)
    if match_fahrenheit:
        temperature_fahrenheit = float(match_fahrenheit.group(1))
        # Convert Fahrenheit to Celsius
        temperature_celsius = convert_from_farhenheit_to_celsius(temperature_fahrenheit)
        return round(temperature_celsius, 2)

    # Search for Kelvin
    match_kelvin = re.search(kelvin_pattern, text)
    if match_kelvin:
        temperature_kelvin = float(match_kelvin.group(1))
        # Convert Kelvin to Celsius
        temperature_celsius = temperature_kelvin - 273.15
        return round(temperature_celsius, 2)

    return None


def check_prefix(prefixes, unit, text):

    # Checks if the prefix that came before the unit is in the dictionary. If it is, we want to multiply the value with the
    # corresponding multiplier.
    if text[text.find(unit) - 1] in prefixes:
        prefix = text[text.find(unit) - 1]
    
    # If text[text.find("pa") - 1] is a letter and it is not in the dictionary, we want to print an error message and return.
    elif text[text.find(unit) - 1].isalpha():
        print(f"Prefix {text[text.find(unit) - 1]} is not supported")
        return "Not supported"
    
    else: 
        prefix = None
    
    return prefix

def extract_pressure_from_string(text):
    """
    Extracts the pressure from a string. The pressure can be given in mmHg, Pa, atm, bar, psi and torr. It can also be given 
    in KPa etc. If the function is unable to extract the pressure, it returns None. 
    """

    # Create a dictionary of the prefixes, where the keys are the prefixes and the values are the corresponding multipliers.
    prefixes = {"k": 1000, "h": 100}

    conversion_to_mmhg = {
    'pa': 0.00750062,  # 1 Pa to mmHg
    'atm': 760,        # 1 atm to mmHg
    'bar': 750.062,    # 1 bar to mmHg
    'torr': 1,         # 1 torr to mmHg (by definition, since 1 torr ≈ 1 mmHg)
    'mmhg': 1,         # 1 mmHg to mmHg
    'psi': 51.7149,    # 1 psi to mmHg
}

    unit = None
    prefix = None 
    value = None

    if "mmhg" in text:
        unit = "mmhg"
        # Checks if the prefix that came before the unit is in the dictionary. If it is, we want to multiply the value with the
        # corresponding multiplier.
        prefix = check_prefix(prefixes, unit, text)

        if prefix == "Not supported":
            return None

    elif "pa" in text:
        unit = "pa"
        prefix = check_prefix(prefixes, unit, text)

        if prefix == "Not supported":
            return None

    elif  "atm" in text: 
        unit = "atm"
        prefix = check_prefix(prefixes, unit, text)

        if prefix == "Not supported":
            return None

    elif "bar" in text:
        unit = "bar"
        prefix = check_prefix(prefixes, unit, text)

        if prefix == "Not supported":
            return None

    elif "torr" in text:
        unit = "torr"
        prefix = check_prefix(prefixes, unit, text)

        if prefix == "Not supported":
            return None

    elif "psi" in text:
        unit = "psi"
        prefix = check_prefix(prefixes, unit, text)

        if prefix == "Not supported":
            return None

    else: 
        print(f"Pressure unit {text} is not supported")
        return None
    
    # Have to identify the numerical value in the string. This is the vapor pressure value. 
    # Identifies if there is a scientific notation in the string.
    scientific_notation_structure = re.compile('[-+]?[\\d]+\\.?[\\d]*[Ee](?:[-+]?[\\d]+)?')
    scientific_notation = re.findall(scientific_notation_structure, text)


    if len(scientific_notation) > 1:
        print(f"More than one scientific notation in {text}")
        return None
    
    elif len(scientific_notation) == 1:
        value = float(scientific_notation[0])

    # If there is no scientific notation in the string, we want to extract the pressure value. However, if the value is inside 
    # a parenthesis we want to ignore this value. 
    else:

        pressure_pattern = re.compile(r"(\d+\.?\d*)")
        pressure_values = re.findall(pressure_pattern, text)

        # If there are more than one pressure value, we want to remove the value inside the parenthesis if there is one and then 
        # do the search again.
        if len(pressure_values) > 1 and "(" in text:
            text = re.sub(r"\(.*?\)", "", text)
            pressure_values = re.findall(pressure_pattern, text)

        # If there are more than one pressure value and no paranthesis, we want to print an error message and return.
        if len(pressure_values) > 1:
            print(f"More than one pressure value in {text}")
            pressure_values = None

        if len(pressure_values) == 1:
            value = float(pressure_values[0])

    if value is not None and prefix is not None:
        value *= prefixes[prefix]

    if value is not None and unit is not None:
        value *= conversion_to_mmhg[unit]

    if value is not None:
        # Want to return the pressure as log(mmHg)
        value = np.log10(value)
        return value
    
    print(f"Unable to extract pressure from {text}")
    return None


def treat_vapor_pressure_string(vapor_pressure_string):
    """
    Have to treat the vapor pressure string in order to extract the pressure and temperature. In other words, we have to get the string 
    on the correct format. 
    """
    # Makes the whole string lower case. For example, "1.2 MM HG" becomes "1.2 mm hg"
    vapor_pressure_string = vapor_pressure_string.lower()

    # Want to remove "[" and "]" from the string. For example, "1.2 [mmHg]" becomes "1.2 mmHg"
    vapor_pressure_string = vapor_pressure_string.replace("[", "")
    vapor_pressure_string = vapor_pressure_string.replace("]", "")

    # Replace @ with "at". For example, "at 25 ºC" becomes "@ 25 ºC"
    vapor_pressure_string = vapor_pressure_string.replace(" at ", "@")

    # Replace deg with °. For example, "at 25 deg C" becomes "at 25 °C"
    vapor_pressure_string = vapor_pressure_string.replace("deg", "°")

    # Remove spaces from the string. For example, "1.2 mm Hg" becomes "1.2mmHg"
    vapor_pressure_string = vapor_pressure_string.replace(" ", "")

    # Replace the comma with a dot. For example, "1,2mmHg" becomes "1.2mmHg"
    vapor_pressure_string = vapor_pressure_string.replace(",", ".")

    # Replace x with an e. For example, "1.2x10-2mmHg" becomes "1.2e10-2mmHg"
    vapor_pressure_string = vapor_pressure_string.replace("x10", "e")

    # Replace = with a :. For example, "1.2mmHg=25°C" becomes "1.2mmHg:25°C"
    vapor_pressure_string = vapor_pressure_string.replace("=", ":")

    return vapor_pressure_string


def extract_pressure_and_temperature_from_string(df, index, vapor_pressure_string, pressure_number):
    """
    Extracts the pressure and temperature from a string. Then saves the pressure and temperature to the dataframe.
    """

    # Sometimes it says that the pressure is < 0.01 mmHg. In those cases, we would like to not add the pressure to the dataframe.
    if "<" not in vapor_pressure_string and ">" not in vapor_pressure_string and "lessthan" not in vapor_pressure_string and "greaterthan" not in vapor_pressure_string:

        # Want to split the text at the "@". This will separate the pressure and the temperature.
        splitted_string = vapor_pressure_string.split("@")

        if len(splitted_string) == 1: 
            # We were not able to split the string at the "@". Try to split at ":" instead.
            splitted_string = vapor_pressure_string.split(":")
            if len(splitted_string) == 1:
                # We were not able to split the string at the ":" either. Try to split at ".".
                splitted_string = vapor_pressure_string.split(".")
                if len(splitted_string) == 1:
                    print(f"Unable to split {vapor_pressure_string} at '@', '.' and ':'")
                    return


        if len(splitted_string) > 2: 
            print(f"More than one '@', ':' or '.' in {vapor_pressure_string}")
            return
        

        # Want to locate which of the strings that contains the temperature and which contains the pressure. If "°" is in 
        # the strings, it is temperature. 
        pressure_string = None
        temperature_string = None
        
        located_temperature = False
        for string in splitted_string:
            if "°" in string: 
                temperature_string = string
                located_temperature = True
            else:
                pressure_string = string


        # If the temperature was not located, we want to print an error message and return.
        if not located_temperature:
            for string in splitted_string:
                save_string_as_pressure = True
                if "k" in string: 
                    # If the element that comes before "k" is a number, we want to add "°" to the string.
                    if string[string.find("k") - 1].isdigit():
                        temperature_string = string
                        temperature_string = temperature_string[:temperature_string.find("k")] + "°" + temperature_string[temperature_string.find("k"):]
                        located_temperature = True
                        save_string_as_pressure = False

                if "c" in string:
                    # If the element that comes before "c" is a number, we want to add "°" to the string.
                    if string[string.find("c") - 1].isdigit():
                        temperature_string = string
                        temperature_string = temperature_string[:temperature_string.find("c")] + "°" + temperature_string[temperature_string.find("c"):]
                        located_temperature = True
                        save_string_as_pressure = False

                if "f" in string:
                    # If the element that comes before "f" is a number, we want to add "°" to the string.
                    if string[string.find("f") - 1].isdigit():
                        temperature_string = string
                        temperature_string = temperature_string[:temperature_string.find("f")] + "°" + temperature_string[temperature_string.find("f"):]
                        located_temperature = True
                        save_string_as_pressure = False

                if save_string_as_pressure:
                    pressure_string = string
                    
            if not located_temperature:
                print(f"Unable to locate temperature in {vapor_pressure_string} as no º was found.")
                return

        try: 
            # If the string is on the form vaporpressure.kpa@25°c:63.7, we must move the number after ":" to the pressure string.
            if temperature_string[temperature_string.find("°") + 2] == ":":
                pressure_string += temperature_string[temperature_string.find("°") + 3:]
                temperature_string = temperature_string[:temperature_string.find("°") + 2]
                located_temperature = True

        except IndexError:
            pass

            
        # Extracts the temperature from the string. 
        temperature = extract_temperature_from_string(temperature_string)

        # Extracts the pressure from the string as log(mmHg).
        pressure = extract_pressure_from_string(pressure_string)

        # If the temperature and pressure are not None, we want to add them to the pressures list
        if temperature and pressure:
            # Add the pressure to the corresponding column
            column_name = f"Pressure {pressure_number + 1} [log(mmHg)]"
            df.at[index, column_name] = f"P({temperature}) = {pressure}"

        elif temperature is None and pressure is None: 
            print(f"Unable to extract temperature and pressure from {vapor_pressure_string}")

        elif temperature is None:
            print(f"Unable to extract temperature from {vapor_pressure_string}")

        elif pressure is None:
            print(f"Unable to extract pressure from {vapor_pressure_string}")

for index, cid_number in enumerate(df["CID"]):

    try: 
        vapor_pressure = extract_vapor_pressure(cid_number)
        if 'Vapor Pressure' in vapor_pressure:
            for vp in vapor_pressure['Vapor Pressure']:
                    # Here we extract the vapor pressure value from the dictionary if it exists
                    if 'Value' in vp and 'StringWithMarkup' in vp['Value'] and vp['Value']['StringWithMarkup']:
                        # The pressure comes as a string when extracted from pubchem. 
                        vapor_pressure_string = vp['Value']['StringWithMarkup'][0]['String']

                        vapor_pressure_string = treat_vapor_pressure_string(vapor_pressure_string)

                        # If there is a ";" in the string, there will be multiple vapor pressures. For example, 
                        # 0.4mmhg@68°f;6mmhg@140°f(ntp.1992) becomes 0.4mmhg@68°f and 6mmhg@140°f(ntp.1992). 
                        multiple_pressure_strings = []
                        if ";" in vapor_pressure_string:
                            # Split the string at the ";" and extract the first vapor pressure
                            multiple_pressure_strings = vapor_pressure_string.split(";")

                            for pressure_number, vapor_pressure_string in enumerate(multiple_pressure_strings):
                                extract_pressure_and_temperature_from_string(df, index, vapor_pressure_string, pressure_number)
                        
                        # If there is no ";" in the string, there is only one vapor pressure value. 
                        else: 
                            extract_pressure_and_temperature_from_string(df, index, vapor_pressure_string, pressure_number)

        if not 'Vapor Pressure' in vapor_pressure:
            df.at[index, "Comment"] = "Vapor pressure data is not available."

        if index % 10 == 0:
            pass
            #df.to_excel('output.xlsx', index=False)

    except UnicodeDecodeError as e:
        print(f"Error decoding response for CAS {cid_number}: {e}")
        df.at[index, "Comment"] = "Error retrieving data"

    except Exception as e:
        print(f"An error occurred for CAS {cid_number}: {e}")
        df.at[index, "Comment"] = "Error retrieving data"

#df.to_excel('output.xlsx', index=False)
#df

Error decoding response for CAS 297: 'utf-8' codec can't decode byte 0x96 in position 665249: invalid start byte
Unable to locate temperature in 0.000217mmhg as no º was found.
Error decoding response for CAS 783: 'utf-8' codec can't decode byte 0x96 in position 419980: invalid start byte
Unable to locate temperature in 0.23mmhg as no º was found.
Unable to locate temperature in 0mmhg(approx)(niosh.2023) as no º was found.
Unable to locate temperature in 0.11mmhg as no º was found.
Unable to locate temperature in 0.0000054mmhg as no º was found.
Pressure unit vaporpressurenegligible is not supported
Unable to extract pressure from vaporpressure@25°c:negligible
Unable to locate temperature in 0mmhg(approx)(niosh.2023) as no º was found.
Unable to split 0mmhg(approx) at '@', '.' and ':'
Unable to split 0mmhg(approx) at '@', '.' and ':'
Unable to locate temperature in 0.00000002mmhg as no º was found.
Unable to locate temperature in 181.0mmhg as no º was found.
Unable to locate temperatur

/var/folders/sb/xp5jwlt55878lf0mfpdf0y2r0000gn/T/ipykernel_12048/3609771794.py:180: RuntimeWarning: divide by zero encountered in log10
  value = np.log10(value)


Unable to split 0mmhg(approx) at '@', '.' and ':'
Unable to split dependsuponthespecificcompound at '@', '.' and ':'
Unable to split 0mmhg(approx) at '@', '.' and ':'
Unable to locate temperature in 0mmhg(approx)(niosh.2023) as no º was found.
Unable to split 0mmhg(approx) at '@', '.' and ':'
Unable to split 0mmhg(approx) at '@', '.' and ':'
More than one scientific notation in 1.47e-6pa(1.10e-8mmhg)
Unable to extract pressure from 1.47e-6pa(1.10e-8mmhg)@400k(127°c)
Unable to locate temperature in 0mmhg(approx)(niosh.2023) as no º was found.
Unable to locate temperature in 0mmhg(approx)(niosh.2023) as no º was found.
Unable to locate temperature in 0mmhg(approx)(niosh.2023) as no º was found.
Unable to locate temperature in 0mmhg(approx)(niosh.2023) as no º was found.
Unable to split approximately0mmhg at '@', '.' and ':'
Unable to split 0mmhg(approx) at '@', '.' and ':'
Unable to split 0mmhg(approx) at '@', '.' and ':'
Unable to locate temperature in 41.3atm as no º was found.
Unable 

,CID,NAME,FORMULA,InChI,SMILES,Comment,Pressure 1 [log(mmHg)],Pressure 2 [log(mmHg)],Pressure 3 [log(mmHg)],Pressure 4 [log(mmHg)],Pressure 5 [log(mmHg)],Pressure 6 [log(mmHg)],Pressure 7 [log(mmHg)],Pressure 9 [log(mmHg)]
0,297,Methane,CH4,InChI=1S/CH4/h1H4,C,Error retrieving data,nan,nan,nan,nan,nan,nan,nan,nan
1,533,Hydrogen selenide,H2Se,InChI=1S/H2Se/h1H2,[SeH2],nan,P(30.8) = 3.959994838328416,P(0.2) = 3.534026106056135,P(21.11) = 3.858537197569639,nan,nan,nan,nan,nan
2,588,creatinine,C4H7N3O,"InChI=1S/C4H7N3O/c1-7-2-3(8)6-4(7)5/h2H2,1H3,(...",CN1CC(=O)N=C1N,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,783,Hydrogen,H2,InChI=1S/H2/h1H,[HH],Error retrieving data,nan,nan,nan,nan,nan,nan,nan,nan
4,807,Iodine,I2,InChI=1S/I2/c1-2,II,nan,nan,nan,P(25.0) = -0.5228787452803376,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4760,154824931,"(8R)-1,5,6,8,9,10,11,11-octachloro-4-oxatetrac...",C10H4Cl8O,"InChI=1S/C10H4Cl8O/c11-3-1-2(6-9(3,16)19-6)8(1...",C12C(C(C3(C1O3)Cl)Cl)C4(C(=C(C2(C4(Cl)Cl)Cl)Cl...,nan,nan,nan,nan,nan,nan,nan,nan,nan
4761,154825517,"(1R,3R,4S,6S,7S)-1,3,4,7,8,9,10,10-octachlorot...",C10H6Cl8,InChI=1S/C10H6Cl8/c11-3-1-2-4(5(3)12)9(16)7(14...,C1C2C(C(C1Cl)Cl)C3(C(=C(C2(C3(Cl)Cl)Cl)Cl)Cl)Cl,nan,nan,nan,nan,nan,nan,nan,nan,nan
4762,154825547,"(4E)-4-ethylidene-7-hydroxy-6,7,14-trimethyl-2...",C19H27NO6,"InChI=1S/C19H27NO6/c1-5-13-10-12(2)19(3,24)18(...",CC=C1CC(C(C(=O)OCC2=CCN(CCC(C2=O)OC1=O)C)(C)O)C,nan,nan,P(25.0) = -13.522878745280337,nan,nan,nan,nan,nan,nan
4763,156619590,"(2R,3aS,5aR,5bS,9S,13S,14R,16aS,16bR)-2-((6-De...",C41H65NO10,InChI=1S/C41H65NO10/c1-10-26-12-11-13-34(52-36...,CCC1CCCC(C(C(=O)C2=CC3C4CC(CC4C=CC3C2CC(=O)O1)...,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [ ]:
# Remove rows where the comment is "Error retrieving data"
df = df[df['Comment'] != 'Error retrieving data']

# Replace nan with np.nan
df.replace(to_replace=['nan'], value=np.nan, inplace=True)

# Remove rows where all the pressures are NaN
df = df.dropna(subset=['Pressure 1 [log(mmHg)]', 'Pressure 2 [log(mmHg)]', 'Pressure 3 [log(mmHg)]', 'Pressure 4 [log(mmHg)]', 'Pressure 5 [log(mmHg)]', 'Pressure 6 [log(mmHg)]', 'Pressure 7 [log(mmHg)]',  'Pressure 9 [log(mmHg)]'], how='all')


# Create a function for shifting values to the first empty cell in a row
def shift_pressures(row):

    # Extract the pressures from the row
    pressures = row[['Pressure 1 [log(mmHg)]', 'Pressure 2 [log(mmHg)]', 
                     'Pressure 3 [log(mmHg)]', 'Pressure 4 [log(mmHg)]', 
                     'Pressure 5 [log(mmHg)]', 'Pressure 6 [log(mmHg)]', 
                     'Pressure 7 [log(mmHg)]', 'Pressure 9 [log(mmHg)]']].tolist()
    
    # Remove NaN values
    pressures = [p for p in pressures if not pd.isna(p)]

    # Update the row with the shifted values
    for i, col in enumerate(['Pressure 1 [log(mmHg)]', 'Pressure 2 [log(mmHg)]', 
                             'Pressure 3 [log(mmHg)]', 'Pressure 4 [log(mmHg)]', 
                             'Pressure 5 [log(mmHg)]', 'Pressure 6 [log(mmHg)]', 
                             'Pressure 7 [log(mmHg)]', 'Pressure 9 [log(mmHg)]']):
        row[col] = pressures[i] if i < len(pressures) else np.nan
    return row

# Apply the function to the dataframe
df = df.apply(shift_pressures, axis=1)

# Remove all the columns with only NaN values
df = df.dropna(axis=1, how='all')

In [117]:
#df.to_excel('output.xlsx', index=False)

Now, we would like to sort the found temperatures in the excel file from lowest to highest. 

In [136]:
import pubchempy as pcp
# Read the Excel file and create a DataFrame
df_2 = pd.read_excel('output.xlsx')

# Function to get Canonical SMILES from CID
def get_canonical_smiles(cid):
    try:
        compound = pcp.Compound.from_cid(cid)
        return compound.canonical_smiles
    except Exception as e:
        print(f"Error retrieving Canonical SMILES for CID {cid}: {str(e)}")
        return None

# Add a new column 'Canonical SMILES' to the DataFrame
df_2['Canonical SMILES'] = df_2['CID'].apply(get_canonical_smiles)

df_2

,CID,NAME,FORMULA,InChI,SMILES,Pressure 1 [log(mmHg)],Pressure 2 [log(mmHg)],Pressure 3 [log(mmHg)],Pressure 4 [log(mmHg)],Pressure 5 [log(mmHg)],Pressure 6 [log(mmHg)],Pressure 7 [log(mmHg)],Canonical SMILES
0,533,Hydrogen selenide,H2Se,InChI=1S/H2Se/h1H2,[SeH2],P(30.8) = 3.959994838328416,P(0.2) = 3.534026106056135,P(21.11) = 3.858537197569639,NaN,NaN,NaN,NaN,[SeH2]
1,807,Iodine,I2,InChI=1S/I2/c1-2,II,P(25.0) = -0.5228787452803376,NaN,NaN,NaN,NaN,NaN,NaN,II
2,947,Nitrogen,N2,InChI=1S/N2/c1-2,N#N,P(-236.0) = -2.1249028364149836,P(-232.0) = -1.1249028364149833,P(-226.8) = -0.1249028364149835,P(-220.2) = 0.8750971635850165,P(-211.1) = 1.8750971635850164,P(-195.9) = 2.8750971635850164,NaN,N#N
3,977,Oxygen,O2,InChI=1S/O2/c1-2,O=O,P(-211.9) = 0.8750971635850165,P(-200.5) = 1.8750971635850164,P(-118.0) = 4.580960875868936,NaN,NaN,NaN,NaN,O=O
4,2879,P-Cresol,C7H8O,"InChI=1S/C7H8O/c1-6-2-4-7(8)5-3-6/h2-5,8H,1H3",CC1=CC=C(C=C1)O,P(20.0) = -1.3979400086720375,P(25.0) = -0.958607314841775,NaN,NaN,NaN,NaN,NaN,CC1=CC=C(C=C1)O
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2353,137705800,"(1R,4S,7R,17S)-7-hydroxy-3',6,7-trimethylspiro...",C18H25NO6,InChI=1S/C18H25NO6/c1-10-8-18(11(2)25-18)16(21...,CC1CC2(C(O2)C)C(=O)OC3CCN4C3C(=CC4)COC(=O)C1(C)O,P(25.0) = -12.031517051446064,NaN,NaN,NaN,NaN,NaN,NaN,CC1CC2(C(O2)C)C(=O)OC3CCN4C3C(=CC4)COC(=O)C1(C)O
2354,146159513,"(4S,7R)-7-hydroxy-3',6,7-trimethylspiro[2,9-di...",C18H25NO6,InChI=1S/C18H25NO6/c1-10-8-18(11(2)25-18)16(21...,CC1CC2(C(O2)C)C(=O)OC3CCN4C3C(=CC4)COC(=O)C1(C)O,P(25.0) = -12.031517051446064,NaN,NaN,NaN,NaN,NaN,NaN,CC1CC2(C(O2)C)C(=O)OC3CCN4C3C(=CC4)COC(=O)C1(C)O
2355,146159675,"[(1R)-1,7,7-trimethyl-2-bicyclo[2.2.1]heptanyl...",C14H22O2,InChI=1S/C14H22O2/c1-9(2)12(15)16-11-8-10-6-7-...,CC(=C)C(=O)OC1CC2CCC1(C2(C)C)C,P(25.0) = -0.958607314841775,NaN,NaN,NaN,NaN,NaN,NaN,CC(=C)C(=O)OC1CC2CCC1(C2(C)C)C
2356,154825547,"(4E)-4-ethylidene-7-hydroxy-6,7,14-trimethyl-2...",C19H27NO6,"InChI=1S/C19H27NO6/c1-5-13-10-12(2)19(3,24)18(...",CC=C1CC(C(C(=O)OCC2=CCN(CCC(C2=O)OC1=O)C)(C)O)C,P(25.0) = -13.522878745280337,NaN,NaN,NaN,NaN,NaN,NaN,CC=C1CC(C(C(=O)OCC2=CCN(CCC(C2=O)OC1=O)C)(C)O)C
